In [5]:
using JuMP, Gurobi, Random, CSV, DataFrames, Statistics, JLD2, FileIO
include("scripts/display.jl")
include("scripts/model.jl");
include("scripts/run.jl");

In [6]:
const dr_speed = 5 # nb of minutes per km when driving;
const wk_speed = 12 # nb of minutes per km when walking;

# Map & ts-network parameters
map_title= "vbs_U30_BIG_cust_C20_BIG"
hubs_locs=[5,14,26];
numlocs=30; 
vbs_ind=1:numlocs;
numcust=10; 
horizon = dr_speed*2500									
tstep = 500;

# Model Inputs
num_veh=5;
G = 1.5 # max driving detour ratio for customers (if W=2: driving time must be inferior to 2*gamma["driving"])
Gtype= "shortestpathpercent" # or "absolutetime"
Wk = 120 # max walking time for customers from origin or to destination
Q= 8 # vehicle capacity;
depot_locs = [3,4,3,4,3]#[rand(1:numlocs) for i in 1:num_veh]

datafolder="data/"*map_title*"/";
map_inputs = (datafolder,hubs_locs,vbs_ind,numlocs,numcust,horizon,tstep)
model_inputs = (G,Gtype,Wk,num_veh,Q,depot_locs);

Over G

In [7]:
coefficients=1,1,1,0.01,1,0;
list_G=[0.5,0.6,0.7,0.8]
res=Dict();
obj_values=Vector();
times=Vector();
for G in list_G
    println("Solving for G=$G")
    model_inputs = (G,Gtype,Wk,num_veh,Q,depot_locs); # Update model_inputs with new G
    map, tsnetwork, params, cust, vbs, arcs, wo, wd, abbrev,shortest_time = create_network(map_inputs, model_inputs);
    q, t, I, K=abbrev;
    xi,x,z,obj,solvetime,objs=network_model_v2(Q,abbrev,wo,tsnetwork,params,coefficients);
    res[G]=Dict("xi"=>xi,"x"=>x,"z"=>z, "obj"=>obj, "solvetime"=>solvetime, "objs"=>objs)
    push!(obj_values,obj)
    push!(times,solvetime)
end

Solving for G=0.5
Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1639020 rows, 617809 columns and 3947869 nonzeros
Model fingerprint: 0x7be30586
Variable types: 0 continuous, 617809 integer (617809 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e-02, 6e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 755937 rows and 100480 columns (presolve time = 5s) ...
Presolve removed 755937 rows and 100480 columns (presolve time = 10s) ...
Presolve removed 761608 rows and 114759 columns (presolve time = 15s) ...
Presolve removed 764004 rows and 150546 columns (presolve time = 20s) ...
Presolve removed 764004 rows and 150546 columns (presolve time = 987s) ...
Presolve removed 990516 rows and 275363 columns (presolve

In [ ]:
plot_obj_time(obj_values,times,[0.5,0.6,0.7,0.8],"G")

Over K

In [ ]:
coefficients=1,1,1,0.01,1,0;
G=1
list_K=[1,2]
res_K=Dict();
obj_values_K=Vector();
times_K=Vector();
for num_veh in list_K
    println("Solving for K=$num_veh")
    depot_locs = [rand(1:numlocs) for i in 1:num_veh]
    model_inputs = (G,Gtype,Wk,num_veh,Q,depot_locs); # Update model_inputs with new num_veh
    map, tsnetwork, params, cust, vbs, arcs, wo, wd, abbrev,shortest_time = create_network(map_inputs, model_inputs);
    q, t, I, K=abbrev;
    xi,x,z,obj,solvetime,objs=network_model_v2(Q,abbrev,wo,tsnetwork,params,coefficients);
    res_K[num_veh]=Dict("xi"=>xi,"x"=>x,"z"=>z, "obj"=>obj, "solvetime"=>solvetime, "objs"=>objs)
    push!(obj_values_K,obj)
    push!(times_K,solvetime)
end

In [ ]:
plot_obj_time(obj_values_K./8,times_K,list_K,"K")